In [331]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import nfl_data_py as nfl
import time
import cfbd
from cfbd.rest import ApiException
from pprintpp import pprint
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [332]:
years = range(2014,2025)

In [333]:
df_defensive = pd.read_csv("https://raw.githubusercontent.com/JoshKepler/Portfolio/main/NFL%20Project/Data/Combined_Defensive_Stats.csv")

In [334]:
df_defensive_career = df_defensive.fillna(0)
df_defensive_career = df_defensive_career.groupby(['player_id','player', 'position']).agg({'team_name':'last', 'Season':'last', 'player_game_count':'sum', 'snap_counts_defense':'sum', 'snap_counts_run_defense':'sum', 'snap_counts_coverage':'sum', 'snap_counts_slot':'sum', 'grades_defense': 'max', 'tackles':'sum', 'assists':'sum', 'missed_tackles': 'sum', 'stops':'sum', 'forced_fumbles':'sum', 'grades_tackle':'max', 'grades_run_defense': 'max', 'total_pressures': 'sum','hurries': 'sum', 'hits':'sum', 'sacks':'sum', 'grades_pass_rush_defense': 'max', 'targets':'sum', 'batted_passes':'sum', 'interceptions':'sum', 'receptions':'sum', 'yards':'sum', 'yards_after_catch':'sum', 'touchdowns':'sum', 'qb_rating_against': 'mean', 'grades_coverage_defense':'max'})

seasons_played = df_defensive.groupby(['player_id','player', 'position']).agg({'Season':'count'})

df_defensive_career['seasons_played'] = seasons_played['Season']
df_defensive_career['draft_class'] = df_defensive_career['Season']+1
df_defensive_career = df_defensive_career.reset_index()
df_defensive_career['player'] = df_defensive_career['player'].str.replace(r'\.?\s*Jr\.?', '', regex=True)
df_defensive_career['player_key'] = df_defensive_career['player'] + '_' + df_defensive_career['team_name'].str.lower()

df_defensive_career

,player_id,player,position,team_name,Season,player_game_count,snap_counts_defense,snap_counts_run_defense,snap_counts_coverage,snap_counts_slot,...,interceptions,receptions,yards,yards_after_catch,touchdowns,qb_rating_against,grades_coverage_defense,seasons_played,draft_class,player_key
0,9436,Dante Fowler,ED,FLORIDA,2014,12,665,281,59,21,...,0,2,2,3,0,70.1,71.6,1.0,2015,Dante Fowler_florida
1,9439,Leonard Williams,DI,USC,2014,13,857,330,0,0,...,1,0,0,0,0,0.0,68.8,1.0,2015,Leonard Williams_usc
2,9441,Vic Beasley,ED,CLEMSON,2014,13,561,238,8,0,...,0,1,-5,3,0,79.2,59.8,1.0,2015,Vic Beasley_clemson
3,9444,Trae Waynes,CB,MICH STATE,2014,13,734,266,453,22,...,3,30,446,171,1,60.4,79.4,1.0,2015,Trae Waynes_mich state
4,9445,Danny Shelton,DI,WASHINGTON,2014,14,903,363,3,0,...,0,0,0,0,0,0.0,78.9,1.0,2015,Danny Shelton_washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24160,182912,Chris Peal,CB,GEORGIA,2023,1,6,5,1,1,...,0,0,0,0,0,0.0,60.0,1.0,2024,Chris Peal_georgia
24161,182947,Unknown Temple D99,ED,TEMPLE,2023,1,6,3,0,0,...,0,0,0,0,0,0.0,0.0,1.0,2024,Unknown Temple D99_temple
24162,183045,Joshua Valdez-Alaniz,ED,MARSHALL,2023,1,7,0,0,0,...,0,0,0,0,0,0.0,0.0,1.0,2024,Joshua Valdez-Alaniz_marshall
24163,183052,Marlon McClendon,CB,GA STATE,2023,1,13,2,11,1,...,0,0,0,0,0,0.0,46.9,1.0,2024,Marlon McClendon_ga state


In [335]:
df_defensive_last_season = df_defensive.fillna(0)
df_defensive_last_season = df_defensive_career.groupby(['player_id','player']).last()
df_defensive_last_season = df_defensive_last_season.reset_index()

df_defense_players = df_defensive_career.merge(df_defensive_last_season, on='player_id', suffixes=('','_last_season'))
df_defense_players = df_defense_players.drop(['seasons_played_last_season', 'draft_class_last_season', 'player_key_last_season'], axis=1)
df_defense_players = df_defense_players.groupby(['player_id']).last()
df_defense_players = df_defense_players.reset_index()
df_defense_players

,player_id,player,position,team_name,Season,player_game_count,snap_counts_defense,snap_counts_run_defense,snap_counts_coverage,snap_counts_slot,...,grades_pass_rush_defense_last_season,targets_last_season,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season
0,9436,Dante Fowler,ED,FLORIDA,2014,12,665,281,59,21,...,81.9,3,2,0,2,2,3,0,70.1,71.6
1,9439,Leonard Williams,DI,USC,2014,13,857,330,0,0,...,65.7,0,3,1,0,0,0,0,0.0,68.8
2,9441,Vic Beasley,ED,CLEMSON,2014,13,561,238,8,0,...,88.4,1,2,0,1,-5,3,0,79.2,59.8
3,9444,Trae Waynes,CB,MICH STATE,2014,13,734,266,453,22,...,52.8,59,0,3,30,446,171,1,60.4,79.4
4,9445,Danny Shelton,DI,WASHINGTON,2014,14,903,363,3,0,...,67.3,0,0,0,0,0,0,0,0.0,78.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20518,182912,Chris Peal,CB,GEORGIA,2023,1,6,5,1,1,...,0.0,0,0,0,0,0,0,0,0.0,60.0
20519,182947,Unknown Temple D99,ED,TEMPLE,2023,1,6,3,0,0,...,47.7,0,0,0,0,0,0,0,0.0,0.0
20520,183045,Joshua Valdez-Alaniz,ED,MARSHALL,2023,1,7,0,0,0,...,53.2,0,0,0,0,0,0,0,0.0,0.0
20521,183052,Marlon McClendon,CB,GA STATE,2023,1,13,2,11,1,...,0.0,0,0,0,0,0,0,0,0.0,46.9


In [336]:
combine_df  = nfl.import_combine_data(years)
draft_df = nfl.import_draft_picks(years)

In [337]:
combine_df['player_name'] = combine_df['player_name'].str.replace(r'\.?\s*Jr\.?', '', regex=True)
combine_df['player_key'] = combine_df['player_name'] + '_' + combine_df['school'].str.lower()

combine_df.sort_values('draft_ovr', ascending=True)

,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle,player_key
6075,2018,2018.0,Cleveland Browns,1.0,1.0,MayfBa00,baker-mayfield-1,Baker Mayfield,QB,Oklahoma,6-1,215.0,4.84,NaN,29.0,111.0,7.00,4.28,Baker Mayfield_oklahoma
7647,2022,2022.0,Jacksonville Jaguars,1.0,1.0,WalkTr03,travon-walker-1,Travon Walker,DE,Georgia,6-5,272.0,4.51,NaN,35.5,123.0,6.89,4.32,Travon Walker_georgia
7129,2021,2021.0,Jacksonville Jaguars,1.0,1.0,LawrTr00,trevor-lawrence-1,Trevor Lawrence,QB,Clemson,6-5,213.0,NaN,NaN,NaN,NaN,NaN,NaN,Trevor Lawrence_clemson
6597,2020,2020.0,Cincinnati Bengals,1.0,1.0,BurrJo01,joe-burrow-1,Joe Burrow,QB,LSU,6-3,221.0,NaN,NaN,NaN,NaN,NaN,NaN,Joe Burrow_lsu
7995,2023,2023.0,Carolina Panthers,1.0,1.0,YounBr01,bryce-young-1,Bryce Young,QB,Alabama,5-10,204.0,NaN,NaN,NaN,NaN,NaN,NaN,Bryce Young_alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8315,2024,NaN,None,NaN,NaN,None,roman-wilson-1,Roman Wilson,WR,Michigan,5-11,185.0,4.39,12.0,NaN,NaN,NaN,NaN,Roman Wilson_michigan
8316,2024,NaN,None,NaN,NaN,None,mekhi-wingo-1,Mekhi Wingo,DT,LSU,6-0,284.0,4.85,25.0,31.5,109.0,NaN,NaN,Mekhi Wingo_lsu
8317,2024,NaN,None,NaN,NaN,None,xavier-worthy-1,Xavier Worthy,WR,Texas,5-11,165.0,4.21,NaN,41.0,131.0,NaN,NaN,Xavier Worthy_texas
8318,2024,NaN,None,NaN,NaN,None,jaylen-wright-1,Jaylen Wright,RB,Tennessee,5-11,210.0,4.38,NaN,38.0,134.0,NaN,NaN,Jaylen Wright_tennessee


In [338]:
prospect_df = combine_df.merge(df_defense_players, on='player_key')

defensive_pos = ['CB', 'ED', 'LB', 'DI', 'S']
prospect_df = prospect_df[prospect_df['position'].isin(defensive_pos)]
prospect_df['pos'] = prospect_df['position']

In [339]:
prospect_df = prospect_df[prospect_df['season'] != 2024]
prospect_df['draft_ovr'] = prospect_df['draft_ovr'].fillna(256)
prospect_df['draft_round'] = prospect_df['draft_round'].fillna(8)
prospect_df['draft_team'] = prospect_df['draft_team'].fillna('Undrafted')
prospect_df['draft_year'] = prospect_df['season']

In [340]:
prospect_cb_df = prospect_df[prospect_df['pos'] == 'CB']
prospect_ed_df = prospect_df[prospect_df['pos'] == 'ED']
prospect_lb_df = prospect_df[prospect_df['pos'] == 'LB']
prospect_di_df = prospect_df[prospect_df['pos'] == 'DI']
prospect_s_df = prospect_df[prospect_df['pos'] == 'S']

pos_df_dict = {'prospect_cb_df':prospect_cb_df , 'prospect_ed_df':prospect_ed_df, 'prospect_lb_df':prospect_lb_df, 'prospect_di_df':prospect_di_df, 'prospect_s_df':prospect_s_df}

In [341]:
var_stats = ['wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle','grades_defense', 'tackles',
       'assists', 'missed_tackles', 'stops', 'forced_fumbles', 'grades_tackle',
       'grades_run_defense', 'total_pressures', 'hurries', 'hits', 'sacks',
       'grades_pass_rush_defense', 'targets', 'batted_passes', 'interceptions',
       'receptions', 'yards', 'yards_after_catch', 'touchdowns',
       'qb_rating_against', 'grades_coverage_defense', 'grades_defense_last_season', 'tackles_last_season',
       'assists_last_season', 'missed_tackles_last_season',
       'stops_last_season', 'forced_fumbles_last_season',
       'grades_tackle_last_season', 'grades_run_defense_last_season',
       'total_pressures_last_season', 'hurries_last_season',
       'hits_last_season', 'sacks_last_season',
       'grades_pass_rush_defense_last_season', 'targets_last_season',
       'batted_passes_last_season', 'interceptions_last_season',
       'receptions_last_season', 'yards_last_season',
       'yards_after_catch_last_season', 'touchdowns_last_season',
       'qb_rating_against_last_season', 'grades_coverage_defense_last_season']

for stat in var_stats:
    prospect_ed_df[stat] = prospect_ed_df[stat].fillna(prospect_s_df[stat].mean())

prospect_ed_df

C:\Users\jumpi\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,...,grades_pass_rush_defense_last_season,targets_last_season,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season
7,2015,2015,Undrafted,8.0,256.0,BarnTa00,tavaris-barnes-1,Tavaris Barnes,ED,Clemson,...,68.1,0,1,0,0,0,0,0,0.00,68.6
8,2015,2015,Atlanta Falcons,1.0,8.0,BeasVi00,vic-beasley-1,Vic Beasley,ED,Clemson,...,88.4,1,2,0,1,-5,3,0,79.20,59.8
12,2015,2015,Seattle Seahawks,2.0,63.0,ClarFr01,frank-clark-1,Frank Clark,ED,Michigan,...,90.8,1,3,0,1,-1,1,0,79.20,67.4
17,2015,2015,Undrafted,8.0,256.0,CrawCo00,corey-crawford-1,Corey Crawford,ED,Clemson,...,53.5,1,1,0,1,3,8,0,79.20,64.3
21,2015,2015,New England Patriots,7.0,253.0,DickXz00,xzavier-dickson-1,Xzavier Dickson,ED,Alabama,...,66.3,2,0,0,2,13,10,0,93.80,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,2023,2023,Undrafted,8.0,256.0,RedmJa01,jalen-redmond-1,Jalen Redmond,ED,Oklahoma,...,61.4,0,0,0,0,0,0,0,0.00,60.6
936,2023,2023,Philadelphia Eagles,1.0,30.0,SmitNo02,nolan-smith-1,Nolan Smith,ED,Georgia,...,79.1,14,1,1,11,117,54,0,75.90,72.7
939,2023,2023,Los Angeles Chargers,2.0,54.0,TuipTu00,tuli-tuipulotu-1,Tuli Tuipulotu,ED,USC,...,81.0,5,3,0,5,35,23,0,96.55,67.0
950,2023,2023,Green Bay Packers,4.0,116.0,WoodCo01,colby-wooden-1,Colby Wooden,ED,Auburn,...,71.8,0,5,0,0,0,0,0,0.00,75.6


In [342]:
X = prospect_ed_df[var_stats]
Y = prospect_ed_df['draft_ovr']

In [343]:
model = LinearRegression()
model.fit(X,Y)
y_pred = model.predict(X)
prospect_ed_df['y_pred'] = y_pred
prospect_ed_df.sort_values(by='y_pred').head(15)

C:\Users\jumpi\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,...,targets_last_season,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season,y_pred
217,2017,2017,Cleveland Browns,1.0,1.0,GarrMy00,myles-garrett-1,Myles Garrett,ED,Texas A&M,...,2,3,1,1,13,11,0,39.600000,72.2,-54.298350
628,2020,2020,Washington Football Team,1.0,2.0,YounCh04,chase-young-1,Chase Young,ED,Ohio State,...,4,6,0,4,10,11,0,54.166667,68.5,-14.453552
420,2019,2019,Oakland Raiders,1.0,4.0,FerrCl00,clelin-ferrell-1,Clelin Ferrell,ED,Clemson,...,9,3,0,7,64,49,0,85.166667,73.6,-2.547956
97,2016,2016,San Diego Chargers,1.0,3.0,BosaJo00,joey-bosa-1,Joey Bosa,ED,Ohio State,...,6,1,1,4,41,18,0,66.300000,65.9,9.946488
388,2019,2019,Jacksonville Jaguars,1.0,7.0,AlleJo03,josh-allen-6,Josh Allen,ED,Kentucky,...,51,2,1,37,338,237,1,81.750000,83.2,28.416450
907,2023,2023,Houston Texans,1.0,3.0,AndeWi01,will-anderson-jr-1,Will Anderson,ED,Alabama,...,13,3,1,11,93,78,0,75.733333,70.7,30.283431
8,2015,2015,Atlanta Falcons,1.0,8.0,BeasVi00,vic-beasley-1,Vic Beasley,ED,Clemson,...,1,2,0,1,-5,3,0,79.200000,59.8,34.075001
194,2017,2017,Philadelphia Eagles,1.0,14.0,BarnDe01,derek-barnett-2,Derek Barnett,ED,Tennessee,...,11,2,1,7,43,53,0,59.733333,73.7,34.153945
854,2022,2022,New York Giants,1.0,5.0,ThibKa00,kayvon-thibodeaux-1,Kayvon Thibodeaux,ED,Oregon,...,13,4,0,10,76,56,0,85.600000,76.3,39.625852
562,2020,2020,Pittsburgh Steelers,3.0,102.0,HighAl02,alex-highsmith-1,Alex Highsmith,ED,Charlotte,...,13,5,0,11,96,57,0,94.250000,79.3,46.123351


In [344]:
coefficients = model.coef_
intercept = model.intercept_ 
coefficient_dict = {}

print("Coefficients:")
for i, feature_name in enumerate(X.columns):
    coefficient_dict[feature_name] = coefficients[i]

coefficient_dict['intercept'] = intercept

sorted(coefficient_dict.items(), key=lambda kv_pair: kv_pair[1], reverse=True)

Coefficients:


[('intercept', 755.2095556005671),
 ('forty', 93.04857408529784),
 ('cone', 19.925881674810093),
 ('touchdowns', 19.454393940986154),
 ('touchdowns_last_season', 19.454393940985952),
 ('forced_fumbles_last_season', 1.388747928712423),
 ('forced_fumbles', 1.3887479287118856),
 ('targets_last_season', 1.1760235159298418),
 ('targets', 1.176023515928783),
 ('missed_tackles', 0.7955652424453292),
 ('missed_tackles_last_season', 0.7955652424452451),
 ('hurries', 0.6811852815696238),
 ('hurries_last_season', 0.6811852815678048),
 ('assists_last_season', 0.6750260097339038),
 ('assists', 0.6750260097335317),
 ('stops_last_season', 0.47270089595854614),
 ('stops', 0.47270089595803455),
 ('yards_after_catch_last_season', 0.43803945363357144),
 ('yards_after_catch', 0.43803945363109165),
 ('grades_coverage_defense_last_season', 0.41622651262870214),
 ('grades_coverage_defense', 0.41622651262713806),
 ('grades_tackle_last_season', 0.3860607873968256),
 ('grades_tackle', 0.3860607873962465),
 ('hi